In [0]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

Загружаем фичи

In [0]:
data_root = "/content/drive/My Drive/antifraud/agg"

In [0]:
interval = pd.DataFrame()
agg = pd.DataFrame()
count = pd.DataFrame()
mcc = pd.DataFrame()
time = pd.DataFrame()

In [4]:
for f in tqdm(os.listdir(data_root)):
  if "agg_features" in f:
    agg = pd.concat((agg, pd.read_csv(os.path.join(data_root, f))))
  elif "mcc_features" in f:
    mcc = pd.read_csv(os.path.join(data_root, f))
  elif "time_features" in f:
    time = pd.concat((time, pd.read_csv(os.path.join(data_root, f))))
  elif "intervals" in f:
    interval = pd.read_csv(os.path.join(data_root, f))
  elif "count_features" in f:
    count = pd.read_csv(os.path.join(data_root, f))

  3%|▎         | 4/115 [00:00<00:22,  4.98it/s]/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
100%|██████████| 115/115 [01:30<00:00,  1.27it/s]


In [10]:
agg

,user_id,event_type count CARD_PIN_CHANGE,event_type count CLIENT_DEFINED,event_type count DEPOSIT,event_type count PAYMENT,event_type count UPDATE_USER,event_type count VIEW_STATEMENT,event_type count WITHDRAW,event_type mean CARD_PIN_CHANGE,event_type mean CLIENT_DEFINED,event_type mean DEPOSIT,event_type mean PAYMENT,event_type mean UPDATE_USER,event_type mean VIEW_STATEMENT,event_type mean WITHDRAW,event_type count CARD_PIN_CHANGE distr,event_type count CLIENT_DEFINED distr,event_type count DEPOSIT distr,event_type count PAYMENT distr,event_type count UPDATE_USER distr,event_type count VIEW_STATEMENT distr,event_type count WITHDRAW distr,event_type mean CARD_PIN_CHANGE distr,event_type mean CLIENT_DEFINED distr,event_type mean DEPOSIT distr,event_type mean PAYMENT distr,event_type mean UPDATE_USER distr,event_type mean VIEW_STATEMENT distr,event_type mean WITHDRAW distr,user_defined_event_type count ATM_BALANCE,user_defined_event_type count ATM_CASH,user_defined_event_type count ATM_CASH_CHECK,user_defined_event_type count ATM_MINI_ST,user_defined_event_type count ATM_P2P_CREDIT,user_defined_event_type count ATM_P2P_DEBIT,user_defined_event_type count ATM_PAYMENT,user_defined_event_type count ATM_PHONE,user_defined_event_type count ATM_PIN,user_defined_event_type count ATM_RECEIPT,user_defined_event_type count ATM_SERVICE_ON,...,mcc_group count O distr,mcc_group count Q distr,mcc_group count R distr,mcc_group count T distr,mcc_group count U distr,mcc_group count X distr,mcc_group count Z distr,mcc_group mean A distr,mcc_group mean C distr,mcc_group mean F distr,mcc_group mean H distr,mcc_group mean J distr,mcc_group mean O distr,mcc_group mean Q distr,mcc_group mean R distr,mcc_group mean T distr,mcc_group mean U distr,mcc_group mean X distr,mcc_group mean Z distr,user_defined_event_type count ATM_EDITAUTOPAYMENT,user_defined_event_type mean ATM_EDITAUTOPAYMENT,user_defined_event_type count ATM_EDITAUTOPAYMENT distr,user_defined_event_type mean ATM_EDITAUTOPAYMENT distr,user_defined_event_type count ATM_P2P_CHECK,user_defined_event_type mean ATM_P2P_CHECK,user_defined_event_type count ATM_P2P_CHECK distr,user_defined_event_type mean ATM_P2P_CHECK distr,user_defined_event_type count LOY_BALINQ,user_defined_event_type mean LOY_BALINQ,user_defined_event_type count LOY_BALINQ distr,user_defined_event_type mean LOY_BALINQ distr,mcc_group count S,mcc_group mean S,mcc_group count S distr,mcc_group mean S distr,user_defined_event_type count ATM_SERVICE_OFF,user_defined_event_type mean ATM_SERVICE_OFF,user_defined_event_type count ATM_SERVICE_OFF distr,user_defined_event_type mean ATM_SERVICE_OFF distr,Unnamed: 0
0,100017347,0,0,56,129,2,1,6,0.0,0.0,3006.375000,441.808915,0.0,1.0,13000.000000,0.000000,0.000000,0.288660,0.664948,0.010309,0.005155,0.030928,0.0,0.0,0.182767,0.026859,0.0,0.000061,0.790313,0,16,17,0.0,23,8,12,0.0,0,1,2,...,0.0,0.0,0.525773,0.000000,0.041237,0.082474,0.206186,0.0,0.305348,0.011485,0.00000,0.000000,0.0,0.0,0.015133,0.000000,0.345820,0.008501,0.313713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10003418,0,0,14,77,0,0,5,0.0,0.0,2015.071429,1582.895195,0.0,0.0,20960.000000,0.000000,0.000000,0.145833,0.802083,0.000000,0.000000,0.052083,0.0,0.0,0.082054,0.064455,0.0,0.000000,0.853491,0,0,0,0.0,14,20,3,0.0,0,0,0,...,0.0,0.0,0.395833,0.000000,0.208333,0.000000,0.052083,0.0,0.064221,0.023237,0.00000,0.000000,0.0,0.0,0.051474,0.000000,0.107156,0.000000,0.753912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100115786,0,6,172,390,0,1,22,0.0,0.0,2070.444477,721.824179,0.0,1.0,3568.181818,0.000000,0.010152,0.291032,0.659898,0.000000,0.001692,0.037225,0.0,0.0,0.325467,0.113468,0.0,0.000157,0.560907,0,22,23,0.0,127,69,58,0.0,0,1,0,...,0.0,0.0,0.480541,0.011844,0.101523,0.010152,0.115059,0.0,0.211331,0.052522,0.00012,0.000000,0.0,0.0,0.060543,0.008569,0.316089,0.009157,0.341670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

Берем таргет

In [0]:
path_root_agg = "/content/drive/My Drive/antifraud/trans_april_may_st3_agg"

In [0]:
tr_data = pd.DataFrame()

In [0]:
files = os.listdir(path_root_agg)
files.sort()

In [8]:
for f in tqdm(files[1:]):
  tr_data = tr_data.append(pd.read_parquet(os.path.join(path_root_agg, f)))

100%|██████████| 200/200 [02:42<00:00,  1.23it/s]


In [0]:
path_root_trans = "/content/drive/My Drive/antifraud/trans_april_may_st0_markup_incoming_transactions_anon"

In [0]:
data_trans = pd.DataFrame()

In [0]:
files = os.listdir(path_root_trans)
files.sort()

In [12]:
for f in tqdm(files[1:]):
  data_trans = data_trans.append(pd.read_parquet(os.path.join(path_root_trans, f)))

100%|██████████| 7/7 [00:00<00:00, 18.01it/s]


In [0]:
ids1 = set(tr_data.event_id)
ids2 = set(data_trans.event_id)
target_ids = ids1.intersection(ids2)

In [0]:
target = tr_data[tr_data.event_id.isin(target_ids)][["event_id", "USER_ID",	"USER_ID_RCVR",	"CLASS"]]

In [0]:
interval.set_index("user_id", inplace=True)
time.set_index("user_id", inplace=True)
count.set_index("Unnamed: 0", inplace=True)
mcc.set_index("Unnamed: 0", inplace=True)

In [0]:
interval.index = interval.index.astype(str)
time.index = time.index.astype(str)
count.index = count.index.astype(str)
mcc.index = mcc.index.astype(str)

Бахнем тф идф

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer

In [0]:
agg.user_id = agg.user_id.astype(str)

In [0]:
agg = agg.drop(agg.columns[agg.isna().sum()>1], axis = 1).dropna()

In [0]:
agg.set_index('user_id', inplace=True)

In [0]:
col = [" ".join(c) for c in map(lambda x: x.split(), agg.columns) if c[-1]!='distr' and c[1]=='count']

In [0]:
tfidf = TfidfTransformer()

In [0]:
to_convert = agg[col]

In [0]:
converted = tfidf.fit_transform(to_convert)

In [0]:
converted = pd.DataFrame(converted.toarray()).set_index(agg.index)
converted.columns = col

In [0]:
agg = pd.concat((agg.drop(col, axis = 1), converted), axis = 1)

Склеим все таблицы

In [0]:
ind1 = set(agg.index.astype(str))
ind2 = set(mcc.index.astype(str))
ind3 = set(interval.index.astype(str))
ind4 = set(time.index.astype(str))
#ind5 = set(count.index.astype(str))

ind = set.intersection(ind1, ind2, ind3, ind4)

In [28]:
len(ind)

599832

In [0]:
data = pd.concat((agg.loc[ind], mcc.loc[ind].drop_duplicates(), interval.loc[ind], time.loc[ind]), axis = 1)

In [30]:
data

,event_type mean CARD_PIN_CHANGE,event_type mean CLIENT_DEFINED,event_type mean DEPOSIT,event_type mean PAYMENT,event_type mean UPDATE_USER,event_type mean VIEW_STATEMENT,event_type mean WITHDRAW,event_type count CARD_PIN_CHANGE distr,event_type count CLIENT_DEFINED distr,event_type count DEPOSIT distr,event_type count PAYMENT distr,event_type count UPDATE_USER distr,event_type count VIEW_STATEMENT distr,event_type count WITHDRAW distr,event_type mean CARD_PIN_CHANGE distr,event_type mean CLIENT_DEFINED distr,event_type mean DEPOSIT distr,event_type mean PAYMENT distr,event_type mean UPDATE_USER distr,event_type mean VIEW_STATEMENT distr,event_type mean WITHDRAW distr,user_defined_event_type mean ATM_BALANCE,user_defined_event_type mean ATM_CASH,user_defined_event_type mean ATM_CASH_CHECK,user_defined_event_type mean ATM_P2P_CREDIT,user_defined_event_type mean ATM_P2P_DEBIT,user_defined_event_type mean ATM_PAYMENT,user_defined_event_type mean ATM_PIN,user_defined_event_type mean ATM_RECEIPT,user_defined_event_type mean ATM_SERVICE_ON,user_defined_event_type mean ATM_ST,user_defined_event_type mean ATM_WITHDRAW,user_defined_event_type mean EPOS_PRE_PURCHASE,user_defined_event_type mean EPOS_PURCHASE,user_defined_event_type mean EPOS_RETURN,user_defined_event_type mean GEN_TPIN,user_defined_event_type mean POS_BALANCE,user_defined_event_type mean POS_CARDLESS,user_defined_event_type mean POS_PAYMENT,user_defined_event_type mean POS_PRE_PURCHASE,...,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,fraud interval,genue interval,mu,std
60390758,0.0,0.0,23931.365918,1014.190621,0.0,0.0,33152.380952,0.0,0.000000,0.059250,0.915357,0.000000,0.000000,0.025393,0.0,0.0,0.411914,0.017457,0.0,0.0,0.570629,0.0,9783.333333,1.0,30106.241351,6777.883721,469.473684,0.0,0.0,0.0,0.0,33152.380952,365.000000,148.833333,0.0,0.0,0.0,0.0,0.0,0.0,...,0.318044,1.334576,0.463243,-0.509643,-0.445705,-0.331594,0.391708,0.415239,-1.327641,0.479241,-0.773160,-0.484719,0.518834,-0.675583,0.262360,0.272822,1.531417,-0.119925,-0.910475,-0.639491,-0.116544,-0.841737,-0.769446,-1.024701,0.499490,0.220462,0.494224,0.193773,-1.284499,-1.527327,-0.281901,0.929823,0.088393,-0.589450,-0.407273,0.163520,0.980653,0.970979,0.229492,0.938081
49885074,0.0,0.0,10875.800000,4703.417755,0.0,0.0,7833.333333,0.0,0.000000,0.250000,0.735000,0.000000,0.000000,0.015000,0.0,0.0,0.464529,0.200893,0.0,0.0,0.334578,0.0,6708.333333,1.0,17818.384615,7796.642857,373.934000,0.0,0.0,0.0,0.0,7833.333333,300.476250,7928.367660,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.155038,-0.412972,-0.047356,0.100305,-0.522680,0.032819,0.305726,-0.036703,0.185721,-0.027130,0.550956,-0.315360,-0.027087,0.235733,-0.178046,-0.071114,-0.216477,0.441819,0.133195,0.349334,-0.201255,0.064173,0.252341,-0.352722,0.167041,0.011866,-0.164271,-0.184943,0.155926,-0.251808,-0.085423,-0.072102,0.100999,-0.144428,0.348442,0.278606,0.790000,0.845000,-0.533117,1.120998
75201745,0.0,0.0,6377.695652,1048.679636,0.0,0.0,0.000000,0.0,0.000000,0.172932,0.827068,0.000000,0.000000,0.000000,0.0,0.0,0.858790,0.141210,0.0,0.0,0.000000,0.0,12681.818182,1.0,7176.000000,1907.692308,750.000000,0.0,0.0,0.0,0.0,0.000000,2012.891667,335.612791,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.214770,0.234266,-0.117638,-0.001559,0.154651,-0.270421,0.228910,-0.063053,-0.019779,0.126393,-0.096347,0.225409,0.348802,-0.151236,-0.093905,-0.004711,0.099791,-0.133842,0.106471,0.074223,-0.086728,-0.124630,-0.443984,-0.269230,0.170700,-0.279508,-0.105363,-0.047293,0.031190,-0.082008,-0.415573,0.135077,-0.235697,-0.019314,-0.030412,0.152950,0.827068,0.879699,0.543807,1.258308
69163017,0.0,0.0,600.600000,493.420877,0.0,0.0,2300.000000,0.0,0.064286,0.107143,0.814286,0.000000,0.000000,0.014286,0.0,0.0,0.176958,0.145379,0.0,0.0,0.677662,0.0,1400.000000,1.0,486.428571,1660.746500,162.000000,0.0,0.0,0.0,0.0,2300.000000,54.666667,367.166667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.116196,0.057388,-0.181101,0.255485,-0.134396,0.102246,0.37526

In [31]:
data.to_csv("/content/drive/My Drive/antifraud/agg/training1.csv")

KeyboardInterrupt: ignored

Теперь другие таблички

In [32]:
target

,event_id,USER_ID,USER_ID_RCVR,CLASS
0,845f9da6751a47a28407ae934fe3c7da,100047377,90581096,0
1,d268a6452b4f47fb9c6e161dbd4b1d9d,100093853,51785703,0
2,32e2ca3fad5f4ae89aa51be508be0a03,100241756,14041952,0
5,80608b0f3a3b4e20acad3e923e632119,100445075,103157424,0
6,09d73e640c6f4ab99d25fd04986b09b2,100445075,60381182,0
...,...,...,...,...
1976,bd138f29ec544e7b9bf2178246ab0e0f,VSP25644210,VSP20227278,0
1977,8db26b07f03743ed940ca6ef3aba32ba,VSP297074,53826758,0
1978,87c21609ccbe479eba9e49dd3af25722,VSP32130102,54264326,0
1979,e7ccd23f1b2d4060926a9bf1422bb9e0,VSP32372012,47093731,0


In [33]:
data_trans

,event_id,user_id,user_id_rcvr,policy_action,risk_score,amount_original,event_time_
0,83cfb3a65bd943f18c4da6dc653ccf68,1870825,5540217,REVIEW,684,120000,1.554394e+09
1,a9cc86a4e52a4d898e6f593f50515be0,23465860,26903133,REVIEW,967,16000,1.554463e+09
2,b855e9868024450593239d34d4610513,MBK9395053,MBK9836211,REVIEW,994,10000,1.554536e+09
3,4514599aa3f54aa497311dc9289dfd2d,37570001,37140634,REVIEW,733,340000,1.554549e+09
4,63592180fc8b4a55a3c0fa12d0f79fa4,29705334,53214797,REVIEW,977,10000,1.554810e+09
...,...,...,...,...,...,...,...
14362,fb661f33d21f471fa936599cedbd47ab,49194968,11585469,REVIEW,536,15000,1.555313e+09
14363,d747ed5790554244b2f65972fd5c0059,2026042,22818226,REVIEW,650,8000,1.555915e+09
14364,6bb49760d44c48bfbbfc5378e5167da6,MBK5632283,19364183,REVIEW,644,25000,1.554291e+09
14365,f3d48a2d15c14effb267678b16eba53d,28570186,59430976,REVIEW,796,179000,1.556518e+09


In [0]:
ids = set(target.event_id).intersection(set(data_trans.event_id))

In [0]:
data_trans.set_index('event_id', inplace=True)
target.set_index('event_id', inplace=True)
data_trans = data_trans.loc[ids]

data_trans['class'] = target.loc[ids].CLASS

In [36]:
data_trans

,user_id,user_id_rcvr,policy_action,risk_score,amount_original,event_time_,class
event_id,,,,,,,
edb52e6958d54f53ba737da0c4b4d17a,2676263,58641357,REVIEW,692,65000,1.555322e+09,0
78fe81fb512f4d78a6a2071b9e29e122,25176094,81916544,REVIEW,778,120000,1.556207e+09,0
c65dd6dd795244889798c618f2f51c21,70860625,52599912,REVIEW,567,35000,1.555008e+09,0
c2153ef1a2fc4160ac51f5321245c9d2,18105863,94394637,REVIEW,680,95000,1.554272e+09,0
4fa8d69664854ef382103f4b7233e983,109792460,34840922,REVIEW,727,70000,1.555410e+09,0
...,...,...,...,...,...,...,...
3caf12cd496e45968c4f49a4c9e906ce,60668115,66054958,REVIEW,788,60000,1.554535e+09,0
4cf1fa2b71994f588ce65abf683be807,39755315,107230249,REVIEW,857,58600,1.556024e+09,0
045d5a11009d4e0bac0121d9d10a0686,239018,11591764,REVIEW,829,77000,1.554753e+09,0


In [37]:
(data_trans.user_id.isin(ind)&data_trans.user_id_rcvr.isin(ind)).sum()

214890

In [38]:
data_trans[(data_trans.user_id.isin(ind)&data_trans.user_id_rcvr.isin(ind))]["class"].sum()

4802

In [0]:
data_trans = data_trans[(data_trans.user_id.isin(ind)&data_trans.user_id_rcvr.isin(ind))]

In [0]:
X1 = data.loc[data_trans.user_id].set_index(data_trans.index)
X2 = data.loc[data_trans.user_id_rcvr].set_index(data_trans.index)

X = pd.concat((data_trans, X1, X2), axis = 1)

In [0]:
X = pd.concat((X, count.loc[X.index]), axis = 1)

In [42]:
X

,user_id,user_id_rcvr,policy_action,risk_score,amount_original,event_time_,class,event_type mean CARD_PIN_CHANGE,event_type mean CLIENT_DEFINED,event_type mean DEPOSIT,event_type mean PAYMENT,event_type mean UPDATE_USER,event_type mean VIEW_STATEMENT,event_type mean WITHDRAW,event_type count CARD_PIN_CHANGE distr,event_type count CLIENT_DEFINED distr,event_type count DEPOSIT distr,event_type count PAYMENT distr,event_type count UPDATE_USER distr,event_type count VIEW_STATEMENT distr,event_type count WITHDRAW distr,event_type mean CARD_PIN_CHANGE distr,event_type mean CLIENT_DEFINED distr,event_type mean DEPOSIT distr,event_type mean PAYMENT distr,event_type mean UPDATE_USER distr,event_type mean VIEW_STATEMENT distr,event_type mean WITHDRAW distr,user_defined_event_type mean ATM_BALANCE,user_defined_event_type mean ATM_CASH,user_defined_event_type mean ATM_CASH_CHECK,user_defined_event_type mean ATM_P2P_CREDIT,user_defined_event_type mean ATM_P2P_DEBIT,user_defined_event_type mean ATM_PAYMENT,user_defined_event_type mean ATM_PIN,user_defined_event_type mean ATM_RECEIPT,user_defined_event_type mean ATM_SERVICE_ON,user_defined_event_type mean ATM_ST,user_defined_event_type mean ATM_WITHDRAW,user_defined_event_type mean EPOS_PRE_PURCHASE,...,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,fraud interval,genue interval,mu,std,sum,count,sum.1,count.1
event_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
edb52e6958d54f53ba737da0c4b4d17a,2676263,58641357,REVIEW,692,65000,1.555322e+09,0,0.0,0.0,12454.290556,2370.649782,0.0,0.0,3070.000000,0.0,0.000000,0.129964,0.826715,0.0,0.007220,0.036101,0.0,0.0,0.695967,0.132476,0.0,0.000000,0.171557,0.0,47500.000000,1.0,11376.208387,6887.745246,183.181250,0.0,0.0,0.0,0.0,3070.000000,1434.290000,...,-0.209177,-0.209876,0.152669,0.145871,-0.075189,-0.041432,-0.298215,-0.067188,-0.124317,-0.056888,0.101286,0.030451,0.104534,0.027791,0.066494,-0.093038,0.273685,0.070036,0.003486,-0.247629,0.125808,-0.185785,0.112080,-0.205829,-0.051223,-0.027561,-0.264525,0.159580,-0.061569,0.044977,-0.201356,-0.154256,0.950820,0.942623,0.642705,1.001129,65000,1,65000,1
78fe81fb512f4d78a6a2071b9e29e122,25176094,81916544,REVIEW,778,120000,1.556207e+09,0,0.0,0.0,15080.454937,2536.969104,0.0,0.0,30724.000000,0.0,0.002915,0.153547,0.834791,0.0,0.000972,0.007775,0.0,0.0,0.311957,0.052480,0.0,0.000000,0.635563,0.0,315000.000000,1.0,11381.232987,7757.018366,146.631579,0.0,0.0,0.0,0.0,30724.000000,522.150957,...,-0.020748,-0.305793,-0.018081,0.108598,-0.225191,0.270720,-0.485088,-0.382310,0.159798,-0.217786,0.295361,-0.075551,0.066311,-0.380962,0.040781,0.140886,0.138905,0.149048,0.007571,-0.863167,0.250463,0.156848,-0.479450,-0.476079,-0.140644,0.095653,-0.408813,0.469121,-0.304418,-0.260830,-0.127863,-0.315596,0.901083,0.862094,0.380243,1.230964,1020000,3,120000,1
c65dd6dd795244889798c618f2f51c21,70860625,52599912,REVIEW,567,35000,1.555008e+09,0,0.0,0.0,16339.454545,3608.685517,0.0,0.0,0.000000,0.0,0.000000,0.174051,0.825949,0.0,0.000000,0.000000,0.0,0.0,0.819097,0.180903,0.0,0.000000,0.000000,0.0,27450.000000,1.0,23310.000000,14215.849057,677.181818,0.0,0.0,0.0,0.0,0.000000,0.000000,...,-0.062923,0.149134,-0.031041,-0.040296,-0.108937,0.064755,-0.405728,-0.110684,0.012954,-0.125231,0.068244,-0.170516,-0.121905,0.036499,-0.154093,-0.038117,-0.128727,0.133244,-0.019045,-0.272445,0.063517,-0.044759,-0.272052,0.000980,-0.224051,0.038068,-0.003185,0.123801,0.076721,-0.162935,-0.292043,0.051313,0.859551,0.994382,-0.254871,1.068834,35000,1,35000,1
c2153ef1a2fc4160ac51f5321245c9d2,18105863,94394637,REVIEW,680,95000,1.554272e+09,0,0.0,0.0,89725.176471,8804.644989,0.0,0.0,120263.157895,0.0,0.000000,0.032319,0.895437,0.0,0.000000,0.072243,0.0,0.0,0.410092,0.040242,0.0,0.000000,0.549666,0.0,0.000000,0.0,89725.176471,62866.655172,916.666667,0.0,0.0,0.0,0.0,120263.157895,0.000000,...,-0.007233,0.041763,-0.117171,0.076789,-0.103674,0.084348,-0.121901,-0.

In [0]:
X.to_csv("/content/drive/My Drive/antifraud/agg/train_data.csv")